### 构造可以生成 x state 的变分量子电路

In [1]:
# 导入必要的库
import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

import hamiltonian_until

In [2]:
# 参数设置
steps = 500  # 迭代次数
eta = 0.1  # 学习率
seed = 1024 # 种子数
tol = 1e-8 # 终止条件
deep_layer = 1 # 电路层数

# 损失函数计算损失值的方法，state 模式暂时还未在优化器中跑通
method = 'expval' 

# 量子数
tol_qubits = hamiltonian_until.tol_qubits

# 优化器
# opt = qml.GradientDescentOptimizer(stepsize=0.1)
opt = qml.AdamOptimizer(stepsize=0.1)

np.random.seed(seed)

params = np.random.uniform(low=0, high=2*np.pi, size=(deep_layer * tol_qubits,))

# 需要学习的量子态
min_eigenvector = np.array(hamiltonian_until.min_eigenvector)

In [3]:
dev = qml.device("default.qubit", wires=12)
@qml.qnode(dev)
def variational_circuit_state(weights, deep_layer):

    for idx in range(tol_qubits):
        qml.Hadamard(wires=idx)

    for deep in range(deep_layer):
        # A very minimal variational circuit.
        for k in range(tol_qubits):
            qml.RY(weights[k+deep*tol_qubits], wires=k)
    
        for j in range(tol_qubits-1):
            qml.CNOT(wires=[j, j+1])
        qml.CNOT(wires=[tol_qubits-1, 0])
        
    return qml.state()

# 定义损失函数
def cost(params):
    prepared_state = variational_circuit_state(params, deep_layer)
    return 1 - np.real(np.dot(np.conj(min_eigenvector), prepared_state))

In [4]:
# 优化参数
for i in range(steps):
    params, cost_val = opt.step_and_cost(cost, params)
    if cost_val < tol:
        break
    if i % 10 == 0:
        print(f"Iteration {i}: Cost = {cost_val}")



Iteration 0: Cost = 1.0009798240914645
Iteration 10: Cost = 0.7702673273164435
Iteration 20: Cost = 0.17832537986027797
Iteration 30: Cost = 0.022354701292487622
Iteration 40: Cost = 0.025019031479115328
Iteration 50: Cost = 0.009666111287634616
Iteration 60: Cost = 0.0007904325726005856
Iteration 70: Cost = 0.0016471351136259083
Iteration 80: Cost = 0.00014663745051435306
Iteration 90: Cost = 0.000180068736806982
Iteration 100: Cost = 3.0358920735440442e-05
Iteration 110: Cost = 3.159948328923523e-05
Iteration 120: Cost = 3.169005248881973e-06
Iteration 130: Cost = 2.6865252160668263e-06
Iteration 140: Cost = 4.663294017603903e-07
Iteration 150: Cost = 2.0183057825029493e-07
Iteration 160: Cost = 1.75460929785487e-07
Iteration 170: Cost = 2.549475652813271e-08
Iteration 180: Cost = 1.5373851525168902e-08


In [5]:
print(f"Final cost: {cost_val}")
print(f"Optimized parameters: {params}")

# 打印最终准备的量子态
final_state = variational_circuit_state(params, deep_layer)
print("Prepared Quantum State:")
print(final_state)

Final cost: 6.036235178363825e-09
Optimized parameters: [4.71237303 4.7124589  1.57078406 4.71237222 4.71235171 1.5708085
 1.57080377 1.57074532 1.57081405 4.71238723 1.57089928 4.71242189]
Prepared Quantum State:
[ 1.61630984e-30+0.j -7.41360846e-45+0.j -5.16651746e-31+0.j ...
 -1.80610486e-35+0.j -2.12072612e-40+0.j  5.65027618e-35+0.j]


In [6]:
np.argmax(min_eigenvector)

940

In [7]:
np.argmax(final_state)

940

In [9]:
np.real(np.dot(min_eigenvector, final_state))

0.9999999972791008